# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [35]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
output_data = pd.read_csv("output_data/cities.csv")
dataframe = pd.DataFrame(output_data)
clean_data = dataframe.drop(columns = ["Unnamed: 0"])

In [3]:
drop_na = clean_data.dropna()
drop_na.dtypes

City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
fig = gmaps.figure()
locations = drop_na[["Lat", "Lng"]].astype(float)
humidity = drop_na["Humidity"].astype(float)
heat_layer = gmaps.heatmap_layer(
    locations,
    weights=humidity, 
    dissipating=False,
    max_intensity=100,
    point_radius = 1
)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
ideal_temps = drop_na.loc[(drop_na['Max Temp'] >= 80) & (drop_na['Max Temp'] <= 90)]
ideal_clouds = ideal_temps.loc[ideal_temps['Cloudiness'] <= 30]
vacation_spots = ideal_clouds.loc[ideal_clouds["Humidity"] < 60]
vacation_spots

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
41,Laguna,38.42,-121.42,80.01,19,1,1.97,US,1604098805
132,Makakilo City,21.35,-158.09,84.20,58,20,14.99,US,1604098840
161,‘Ewa Beach,21.32,-158.01,84.20,58,20,14.99,US,1604098848
272,Coihueco,-36.62,-71.83,82.40,28,0,5.82,CL,1604098887
498,Agua Verde,22.90,-106.05,87.80,43,20,11.41,MX,1604098957
526,Poya,-21.35,165.15,83.66,52,0,5.50,NC,1604098965


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [65]:
hotel_df = vacation_spots
hotel_df["Hotel Name"] = ""
hotel_df

/Users/lucykaplan/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
41,Laguna,38.42,-121.42,80.01,19,1,1.97,US,1604098805,
132,Makakilo City,21.35,-158.09,84.20,58,20,14.99,US,1604098840,
161,‘Ewa Beach,21.32,-158.01,84.20,58,20,14.99,US,1604098848,
272,Coihueco,-36.62,-71.83,82.40,28,0,5.82,CL,1604098887,
498,Agua Verde,22.90,-106.05,87.80,43,20,11.41,MX,1604098957,
526,Poya,-21.35,165.15,83.66,52,0,5.50,NC,1604098965,


In [84]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

In [107]:
target_radius = 5000
target_type = "lodging"
params = {
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    response = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.


In [108]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
41,Laguna,38.42,-121.42,80.01,19,1,1.97,US,1604098805,Holiday Inn Express & Suites Elk Grove Central...
132,Makakilo City,21.35,-158.09,84.20,58,20,14.99,US,1604098840,Marriott's Ko Olina Beach Club
161,‘Ewa Beach,21.32,-158.01,84.20,58,20,14.99,US,1604098848,Hampton Inn & Suites Oahu/Kapolei
272,Coihueco,-36.62,-71.83,82.40,28,0,5.82,CL,1604098887,Frutillares Coihueco
498,Agua Verde,22.90,-106.05,87.80,43,20,11.41,MX,1604098957,Laboratorio Prolamar
526,Poya,-21.35,165.15,83.66,52,0,5.50,NC,1604098965,Opt Poya


In [109]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [118]:
# Add marker layer ontop of heat map
hotel_list = hotel_df["Hotel Name"].tolist()
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content = hotel_info
)

fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))